In [ ]:
# ======================
# IMPORTS
# ======================
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date, timedelta

# ======================
# SETTINGS
# ======================
BOT_TOKEN = "8435851436:AAHENY0AGnFImSORLrFl6Mm_kcS8_oyVMDQ"
SPREADSHEET_NAME = "bot"
ADMIN_IDS = [5010534845]  #Telegram ID

# ======================
# GOOGLE SHEETS
# ======================
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    "credentials.json", scope
)
client = gspread.authorize(creds)

sheet = client.open(SPREADSHEET_NAME).worksheet("Students")
groups_sheet = client.open(SPREADSHEET_NAME).worksheet("Groups")

# ======================
# BOT
# ======================
bot = Bot(token=BOT_TOKEN)
dp = Dispatcher(bot)

# ======================
# STORAGE & CONSTANTS
# ======================
selected_students = {}

ACHIEVEMENTS = {
    "3_dz": "3 домашки подряд💫",
    "5_dz": "5 домашек подряд🤌",
    "week_no_skip": "Неделя без пропусков🫡",
    "first_star": "Первая задача со ⭐",
}

# ======================
# KEYBOARDS
# ======================
def student_menu():
    kb = ReplyKeyboardMarkup(resize_keyboard=True)
    kb.add(
        KeyboardButton("📊 Профиль"),
        KeyboardButton("🏅 Достижения")
    )
    kb.add(
        KeyboardButton("👥 Прогресс группы")
    )
    return kb


def admin_menu():
    kb = ReplyKeyboardMarkup(resize_keyboard=True)
    kb.add(
        KeyboardButton("👤 Выбрать ученика"),
        KeyboardButton("➕ ДЗ (+20 XP)")
    )
    kb.add(
        KeyboardButton("🏅 Выдать достижение"),
        KeyboardButton("📊 Статистика группы")
    )
    return kb

# ======================
# HELPERS — STUDENTS
# ======================
def get_student_row(telegram_id: int):
    records = sheet.get_all_records()
    for i, row in enumerate(records, start=2):
        if row["telegram_id"] == telegram_id:
            return i, row
    return None, None


def add_student(telegram_id: int, name: str):
    sheet.append_row([
        telegram_id,
        name,
        "A1",
        0,
        0,
        "",
        ""
    ])


def add_xp(telegram_id: int, amount: int):
    row_num, student = get_student_row(telegram_id)
    if not student:
        return False

    current_xp = int(student["xp"])
    sheet.update_cell(row_num, 4, current_xp + amount)
    return True


def update_streak(telegram_id: int):
    row_num, student = get_student_row(telegram_id)
    if not student:
        return

    today = date.today()
    last_activity = student.get("last_activity")

    if not last_activity:
        new_streak = 1
    else:
        last_date = date.fromisoformat(last_activity)
        if last_date == today:
            return
        elif last_date == today - timedelta(days=1):
            new_streak = int(student["streak"]) + 1
        else:
            new_streak = 1

    sheet.update_cell(row_num, 5, new_streak)
    sheet.update_cell(row_num, 6, today.isoformat())

# ======================
# ACHIEVEMENTS
# ======================
def get_achievements(student):
    raw = student.get("achievements", "")
    if not raw:
        return []
    return raw.split(",")


def add_achievement(telegram_id: int, key: str):
    row_num, student = get_student_row(telegram_id)
    if not student:
        return False

    current = get_achievements(student)
    if key in current:
        return False

    current.append(key)
    sheet.update_cell(row_num, 7, ",".join(current))
    return True

# ======================
# GROUPS
# ======================
def get_group_row(group_name: str):
    records = groups_sheet.get_all_records()
    for i, row in enumerate(records, start=2):
        if row["group"] == group_name:
            return i, row
    return None, None


def add_group_xp(group_name: str, amount: int):
    row_num, group = get_group_row(group_name)
    if not group:
        return

    current_xp = int(group["group_xp"])
    groups_sheet.update_cell(row_num, 2, current_xp + amount)


def get_students_by_group(group_name: str):
    records = sheet.get_all_records()
    return [r for r in records if r["group"] == group_name]


def group_stats(group_name: str, days: int = 7):
    students = get_students_by_group(group_name)
    if not students:
        return None

    today = date.today()
    total = len(students)
    active = 0
    total_streak = 0
    dz_count = 0

    for s in students:
        total_streak += int(s.get("streak", 0))
        last = s.get("last_activity")
        if last:
            last_date = date.fromisoformat(last)
            if (today - last_date).days <= days:
                active += 1
                dz_count += 1

    return {
        "total": total,
        "active": active,
        "dz_count": dz_count,
        "avg_streak": round(total_streak / total, 1)
    }

# ======================
# HANDLERS
# ======================
@dp.message_handler(commands=["start"])
async def start(message: types.Message):
    telegram_id = message.from_user.id
    name = message.from_user.first_name

    _, student = get_student_row(telegram_id)
    if not student:
        add_student(telegram_id, name)

    if telegram_id in ADMIN_IDS:
        await message.answer("Админ-режим", reply_markup=admin_menu())
    else:
        await message.answer("Ты в трекере прогресса", reply_markup=student_menu())


@dp.message_handler(lambda m: m.text == "📊 Профиль")
async def profile(message: types.Message):
    _, student = get_student_row(message.from_user.id)
    if not student:
        return

    await message.answer(
        f"Имя: {student['name']}\n"
        f"XP: {student['xp']}\n"
        f"Серия: {student['streak']} дней"
    )


@dp.message_handler(lambda m: m.text == "👤 Выбрать ученика")
async def choose_student(message: types.Message):
    if message.from_user.id not in ADMIN_IDS:
        return

    kb = ReplyKeyboardMarkup(resize_keyboard=True)
    for r in sheet.get_all_records():
        kb.add(KeyboardButton(f"{r['name']} | {r['telegram_id']}"))
    await message.answer("Выбери ученика:", reply_markup=kb)


@dp.message_handler(lambda m: "|" in m.text)
async def select_student(message: types.Message):
    if message.from_user.id not in ADMIN_IDS:
        return

    try:
        _, telegram_id = message.text.split("|")
        selected_students[message.from_user.id] = int(telegram_id.strip())
        await message.answer("Ученик выбран", reply_markup=admin_menu())
    except:
        pass


@dp.message_handler(lambda m: m.text == "➕ ДЗ (+20 XP)")
async def add_homework(message: types.Message):
    admin_id = message.from_user.id
    if admin_id not in ADMIN_IDS:
        return

    student_id = selected_students.get(admin_id)
    if not student_id:
        await message.answer("Сначала выберете ученика")
        return

    add_xp(student_id, 20)
    update_streak(student_id)

    _, student = get_student_row(student_id)
    add_group_xp(student["group"], 20)

    await message.answer("Учебный шаг зафиксирован. +20 XP")


@dp.message_handler(lambda m: m.text == "🏅 Выдать достижение")
async def give_achievement_menu(message: types.Message):
    if message.from_user.id not in ADMIN_IDS:
        return

    kb = ReplyKeyboardMarkup(resize_keyboard=True)
    for title in ACHIEVEMENTS.values():
        kb.add(KeyboardButton(title))
    await message.answer("Выберете достижение:", reply_markup=kb)


@dp.message_handler(lambda m: m.text in ACHIEVEMENTS.values())
async def give_achievement(message: types.Message):
    admin_id = message.from_user.id
    student_id = selected_students.get(admin_id)
    if not student_id:
        return

    for key, title in ACHIEVEMENTS.items():
        if title == message.text:
            add_achievement(student_id, key)
            await message.answer("Достижение выдано", reply_markup=admin_menu())


@dp.message_handler(lambda m: m.text == "🏅 Достижения")
async def student_achievements(message: types.Message):
    _, student = get_student_row(message.from_user.id)
    if not student:
        return

    ach = get_achievements(student)
    if not ach:
        await message.answer("Достижений пока нет")
        return

    text = "Твои достижения:\n"
    for a in ach:
        text += f"• {ACHIEVEMENTS.get(a, a)}\n"
    await message.answer(text)


@dp.message_handler(lambda m: m.text == "👥 Прогресс группы")
async def group_progress(message: types.Message):
    _, student = get_student_row(message.from_user.id)
    if not student:
        return

    _, group = get_group_row(student["group"])
    xp = int(group["group_xp"])

    filled = min(xp // 50, 10)
    bar = "🟩" * filled + "⬜" * (10 - filled)

    await message.answer(
        f"Группа {student['group']}\n"
        f"{bar}\n"
        f"XP группы: {xp}"
    )


@dp.message_handler(lambda m: m.text == "📊 Статистика группы")
async def admin_stats(message: types.Message):
    admin_id = message.from_user.id
    student_id = selected_students.get(admin_id)
    if not student_id:
        return

    _, student = get_student_row(student_id)
    stats = group_stats(student["group"])

    await message.answer(
        f"Группа {student['group']}\n"
        f"👥 Учеников: {stats['total']}\n"
        f"📘 ДЗ за 7 дней: {stats['dz_count']}\n"
        f"🔥 Средняя серия: {stats['avg_streak']}\n"
        f"⚡ Активных: {stats['active']}"
    )

# ======================
# RUN
# ======================
if __name__ == "__main__":
    executor.start_polling(dp, skip_updates=True)

print('бот запущен')
